# Setup

In [ ]:
# df = pd.read_csv(os.path.join(prep_path, 'df_encoded.csv'), index_col=0)
################# Feature Categorical & Numerical
cateogirical_features_add = ['강남여부', '신축여부','구', '동']
categorical_features = ['k-건설사(시공사)', 'k-관리방식', 'k-난방방식', 'k-단지분류(아파트,주상복합등등)', 'k-복도유형', 'k-사용검사일-사용승인일', 'k-세대타입(분양형태)', 'k-수정일자', 'k-시행사', '경비비관리형태', '관리비 업로드', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '단지신청일', '도로명', '번지', '사용허가여부', '세대전기계약방법',  '아파트명', '청소비관리형태']+cateogirical_features_add

# 시군구 제외
numerical_features_add = ['계약년', '계약월']
numerical_features = ['k-85㎡~135㎡이하', 'k-관리비부과면적', 'k-연면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전용면적별세대현황(60㎡이하)', 'k-전체동수', 'k-전체세대수', 'k-주거전용면적', '건축년도', '건축면적', '계약일', '본번', '부번', '전용면적', '좌표X', '좌표Y', '주차대수', '층']+ numerical_features_add

#assert len(categorical_features) + len(numerical_features) +2== df.shape[1]

# Utils

In [ ]:

from typing import List, Dict
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
import pandas as pd

def concat_train_test(dt, dt_test):
    Utils.remove_unnamed_columns(dt)
    Utils.remove_unnamed_columns(dt_test)
    dt['is_test'] = 0
    dt_test['is_test'] = 1
    dt_test['target'] = 0
    concat = pd.concat([dt, dt_test], axis=0).reset_index(drop=True)
    print(concat['is_test'].value_counts())

    return concat

def unconcat_train_test(concat):
    Utils.remove_unnamed_columns(concat)
    dt = concat.query('is_test==0')
    # y_train = dt['target']
    dt.drop(columns=['is_test'], inplace=True)
    dt_test = concat.query('is_test==1')
    dt_test.drop(columns=['target', 'is_test'], inplace=True)
    return dt, dt_test

# Scaling Numerical Features

In [ ]:


class DataScaler:
    def __init__(self, 
                 target_col: str = None,
                 categorical_cols: List[str] = None,
                 continuous_cols: List[str] = None,
                 exclude_cols: List[str] = None):
        """
        데이터 스케일링을 위한 클래스
        
        Args:
            target_col: 타겟 컬럼명 (스케일링에서 제외)
            categorical_cols: 범주형 변수 리스트
            continuous_cols: 연속형 변수 리스트
            exclude_cols: 스케일링에서 제외할 컬럼 리스트
        """
        self.target_col = target_col
        self.categorical_cols = set(categorical_cols) if categorical_cols else set()
        self.continuous_cols = set(continuous_cols) if continuous_cols else set()
        self.exclude_cols = set(exclude_cols) if exclude_cols else set()
        self.scalers = {}
        
    def _get_appropriate_scaler(self, col_name: str, data: pd.Series) -> object:
        """
        컬럼 특성에 맞는 스케일러 반환
        
        Args:
            col_name: 컬럼명
            data: 스케일링할 데이터
            
        Returns:
            스케일러 객체
        """
        # 범주형 변수는 StandardScaler 사용
        # if col_name in self.categorical_cols:
        #     return StandardScaler()
        
        # 연속형 변수는 왜도에 따라 스케일러 결정
        if abs(data.skew()) > 1:
            # 심한 왜도는 RobustScaler + PowerTransformer
            return Pipeline([
                ('robust', RobustScaler()),
                ('power', PowerTransformer(method='yeo-johnson'))
            ])
        
        # 나머지는 RobustScaler
        return RobustScaler()
        
    def scale_features(self, df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
        """
        특성별 적절한 스케일링 적용
        
        Args:
            df: 스케일링할 데이터프레임
            is_train: 학습 데이터 여부
            
        Returns:
            스케일링된 데이터프레임
        """
        scaled_df = df.copy()
        
        # 스케일링할 컬럼 결정
        if self.continuous_cols or self.categorical_cols:
            # 지정된 컬럼이 있는 경우
            scale_cols = self.continuous_cols | self.categorical_cols
        else:
            # 지정된 컬럼이 없는 경우 수치형 컬럼 자동 선택
            scale_cols = set(df.select_dtypes(include=['int64', 'float64']).columns)
        
        # 제외할 컬럼 처리
        exclude_set = self.exclude_cols.copy()
        if self.target_col:
            exclude_set.add(self.target_col)
        exclude_set.add('is_test')  # 항상 제외
        
        scale_cols = scale_cols - exclude_set
        
        # 실제 존재하는 컬럼만 선택
        actual_scale_cols = scale_cols & set(df.columns)
        
        # 누락된 컬럼 확인
        missing_cols = scale_cols - actual_scale_cols
        if missing_cols:
            print(f"Warning: 다음 컬럼들이 데이터에 없습니다: {sorted(missing_cols)}")
        
        print(f"스케일링 적용 컬럼: {sorted(actual_scale_cols)}")
        
        # 스케일링 수행
        for col in tqdm(actual_scale_cols, desc='Scaling columns'):
            if is_train:
                # 학습 데이터: 스케일러 생성 및 적용
                scaler = self._get_appropriate_scaler(col, scaled_df[col])
                self.scalers[col] = scaler
                scaled_values = scaler.fit_transform(scaled_df[[col]]).ravel()
                scaled_df[col] = scaled_values
            else:
                # 테스트 데이터: 기존 스케일러 적용
                if col in self.scalers:
                    scaled_values = self.scalers[col].transform(scaled_df[[col]]).ravel()
                    scaled_df[col] = scaled_values
                else:
                    print(f"Warning: {col} 컬럼의 스케일러가 없습니다.")
        
        return scaled_df

In [ ]:
df_train, df_test = unconcat_train_test(df) # train, test data 분리

# # categorical, numerical 기준은 baseline 으로 임시 작성. 변경 가능.
# categorical_features =['전용면적', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', '강남여부', '신축여부']
# numerical_features = ['번지', '본번', '부번', '아파트명', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일', '구', '동', '계약년', '계약월']
# add_features = ['구', '동', '계약년', '계약월']
# numerical_features = list(set(numerical_features)-set(add_features))

data_scaler = DataScaler(target_col='target',
                            categorical_cols=categorical_features,
                            continuous_cols=numerical_features,
                            exclude_cols=['is_test']  ) # 아직 

# 1. Numerical Features 들의 경우, outlier removal 없이 이상치에 강건하게 Robust Scaling 을 먼저 적용
# 학습 데이터 전처리
train_scaled = data_scaler.scale_features(df_train[numerical_features], is_train=True)
# 테스트 데이터 전처리 (학습 데이터의 스케일러 사용)
test_scaled = data_scaler.scale_features(df_test[numerical_features], is_train=False)


# 2. Categorical Features 들의 경우, Feature engineering 후 encoding 까지 완료된 이후에 표준화 스케일링을 별도 적용합니다



In [ ]:
train_scaled.isnull().sum()
train_scaled[categorical_features] = df_train[categorical_features]
test_scaled[categorical_features] = df_test[categorical_features]
train_scaled['target'] = df_train['target']
concat_scaled = concat_train_test(train_scaled, test_scaled)

# Label Encoding for Categorical Features

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
def encode_label(dt_train, dt_test, categorical_columns_v2):
    logger.info('#### 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.')
    # 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
    label_encoders = {}

    # Implement Label Encoding
    for col in tqdm( categorical_columns_v2 ):
        lbl = LabelEncoder()
    
        # Label-Encoding을 fit
        lbl.fit( dt_train[col].astype(str) )
        dt_train[col] = lbl.transform(dt_train[col].astype(str))
        label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

        # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
        dt_test[col] = dt_test[col].astype(str)
        for label in np.unique(dt_test[col]):
            if label not in lbl.classes_: # unseen label 데이터인 경우
                lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!
        dt_test[col] = lbl.transform(dt_test[col].astype(str))

        dt_train.head(1)        # 레이블인코딩이 된 모습입니다.

        assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.
    return dt_train, dt_test, label_encoders

In [ ]:
train_encoded, test_encoded, label_encoders = encode_label(df_train[categorical_features], df_test[categorical_features] ,categorical_features)


In [ ]:
train_encoded[numerical_features] = df_train[numerical_features]
train_encoded['target'] = df_train['target']
test_encoded[numerical_features] = df_test[numerical_features]


In [ ]:
train_encoded.shape, test_encoded.shape

df_encoded = concat_train_test(train_encoded, test_encoded)
#df_encoded.to_csv(os.path.join(prep_path, 'df_encoded.csv'), index=False)

df_encoded.isnull().sum()
